<a href="https://colab.research.google.com/github/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Maria_Amado_e_Fernanda_Lisboa_report_handson_5_jupyter_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-on 5: Basic Hybrid Application MPI+OpenMP

M. Amado$^1$, F. Lisboa$^1$

$^1$ Department of Computer Engenier – University SENAI CIMATEC, Salvador, Bahia, Brazil  

# Abstract

A programação paralela possibilita uma redução no tempo de execução dos códigos, e consequentemente, resolução mais rápida de diversos problemas. O modelo híbrido aproveita o compartilhamento de memória entre processos de um mesmo subsistema do OpenMP e a distribuição distinta entre coleções de processos do MPI, podendo ser mais efetivo atualmente do que o uso separado dos dois paradigmas. O presente trabalho visa comparar as estratégias de otimização de um algoritmo de multiplicação de matrizes, considerando a aplicação dos paradigmas, utilizando os modelos com OpenMP, MPI e híbrido. Como já colocado em hipotése, o modelo híbrido apresenta resultados baseados nos valores mais otimizados dos paradigmas executados separadamente, apresentando uma otimização em relação ao tempo sequencial, entretanto também evidenciou o custo da troca de mensagens quando consideramos a paralelização apenas com *threads*. Dessa forma, a abordagem híbrida mostrou-se uma boa alternativa para o maior aproveitamento dos recursos computacionais disponíveis, melhorando tanto o uso da memória compartilhada quando distribuída.

# Introduction

A programação paralela tem como objetivo a melhoria de desempenhos das aplicações a partir da execução de múltiplos processos [1]. possibilita uma redução no tempo de execução dos códigos, e consequentemente, resolução mais rápida de diversos problemas. Dentro desse âmbito há alguns paradigmas, sendo os mais conhecidos memória compartilhada e de memória distribuída entre os processos. Assim, programadores podem  adotar a estratégia da paralelização através da troca de mensagens, indicada para sistemas com memória distribuída, ou a estratégia de paralelização através da criação de threads, voltada para arquiteturas com memória compartilhada [2].

O Paradigma de Memória Compartilhada significa que um único endereço de memória pode ser acessado por cada processador através de procedimentos sincronizados [3]. Nesse contexto, o OpenMP (Open Multi Processing) foi criado para explorar características da arquitetura de memória compartilhada, como o acesso direto à memória através do sistema com baixa latência e muita rapidez.


O Paradigma de Memória Distribuída, se baseia, geralmente, na abordagem "dividir para conquistar" [3]. Dentro desse cenário surge o MPI (Messsage Passing Interface), o qual diversos processos paralelos trabalham concorrentemente em busca de um objetivo comum utilizando "mensagens" entre si [3]. O MPI possui uma coleção de funções, sendo suas principais de envio e recebimento de informações entre os processos. 

Com o avanço dos sistemas computacionais que agora contam com múltiplos processadores e compartilhamento de memória é possível utilizar essas ferramentas em conjunto para implementar programas ainda mais otimizados [2]. Nessas arquiteturas de memória compartilhada e distribuída (híbrida), a combinação da comunicação intra nós do OpenMP com a entre nós do MPI, pode prover uma exploração mais efetiva e moderna de sistemas *distributed shared-memory* (DSM) [3].

A implementação de soluções híbridas, que aproveitam tanto os recursos da memória compartilhada quanto distribuída, tem como tendência a aplicação de um modelo de estrutura hierárquico, o que possibilita tanto a exploração de grãos grandes  e médios com o MPI, quanto do grão fino com o OpenMP, aproveitando, assim, as melhores características dos dois paradigmas [4].

Portanto, esta prática tem como objetivo comparar as estratégias de otimização de um algoritmo de multiplicação de matrizes, considerando a aplicação dos paradigmas, utilizando os modelos com OpenMP, MPI e híbrido.



# Results and Discussion

### Instalação das bibliotecas necessárias

In [ ]:
from IPython.display import clear_output

In [ ]:
!sudo apt install libomp-dev
clear_output(wait=False)

In [ ]:
!sudo apt-get install openmpi-bin
clear_output(wait=False)

### Paralelização utilizando o MPI

Implementação da multiplicação de matrizes com MPI:

#### código

In [ ]:
%%writefile mm-mpi.c

/*
File:           mm-mpi.c
Purpose:        Parallelize matrix multiplication using MPI
Authors:        Francisco Almeida, Domingo Giménez, José Miguel Mantas, Antonio M. Vidal
                'Introducción a la programación paralela,
                 Paraninfo Cengage Learning, 2008, Capítulo 6, 
                 Sección 6.3 Particionado de datos: Código 6.10
                 Multiplicación de matrices por particionado de datos'
Usage:
HowToCompile:   mpicc mm-mpi.c -o mm-mpi
HowToExecute:   mpirun -np <numberOfProcesses> ./mm-mpi <size>
Example:        mpirun -np     4               ./mm-mpi  100
Comments:
                ◆ Spanish code comments;          
*/

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <mpi.h>

// multiplicación de matrices secuencial
// por cada matriz aparece la zona de datos (a, b y c)
// y el número de filas, de columnas y el leading dimension
void mms(double *a, int fa, int ca, int lda, double *b, int fb, int cb, int ldb, double *c, int fc, int cc, int ldc) {
    int i, j, k;
    double s;
    for (i = 0; i < fa; i++) 
        for (j = 0; j < cb; j++) {
            s = 0.;
            for (k = 0; k < ca; k++)
                s += a[i * lda + k] * b[k * ldb + j];
            c[i * ldc + j] = s;
        }
}

// nodo es un identificador del proceso
// y np el número total de procesos
void mm(double *a, int fa, int ca, int lda, double *b, int fb, int cb, int ldb, double *c, int fc, int cc, int ldc, int nodo, int np) {
    int i, j, k;
    double s;
    if (nodo == 0) {
        for (i = 1; i < np; i++)
            MPI_Send(&a[i * lda * fa / np], fa / np * ca, MPI_DOUBLE, i, 20, MPI_COMM_WORLD);
        MPI_Bcast(b, fb * cb, MPI_DOUBLE, 0, MPI_COMM_WORLD);
    } else {
        MPI_Recv(a, fa / np * ca, MPI_DOUBLE, 0, 20, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        MPI_Bcast(b, fb * cb, MPI_DOUBLE, 0, MPI_COMM_WORLD);
    }
    mms(a, fa / np, ca, lda, b, fb, cb, ldb, c, fc / np, cc, ldc);
    if (nodo == 0)
        for (i = 1; i < np; i++)
            MPI_Recv(&c[i * ldc * fc / np],fc / np * cc, MPI_DOUBLE, i, 30, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
    else
        MPI_Send(c, fc / np * cc, MPI_DOUBLE, 0, 30, MPI_COMM_WORLD);
}

/*
c
c initialize - random initialization for array
c
*/

void initialize(double *m, int f, int c, int ld) {
  int i, j;

  for (i = 0; i < f; i++) {
    for (j = 0; j < c; j++) {  
      m[i * ld + j] = (double)(i + j);
    }
  }
}

void initializealea(double *m, int f, int c, int ld) {
  int i, j;

  for (i = 0; i < f; i++) {
    for (j = 0; j < c; j++) {  
      m[i * ld + j] = (double)rand() / RAND_MAX;
    }
  }
}

void escribir(double *m, int f, int c, int ld) {
  int i, j;

  for (i = 0; i < f; i++) {
    for (j = 0; j < c; j++) {  
      printf("%.4lf ",m[i * ld + j]);
    }
    printf("\n");
  }
}

void comparar(double *m1, int fm1, int cm1, int ldm1, double *m2, int fm2, int cm2, int ldm2)
{
  int i, j;

  for(i = 0; i < fm1; i++)
    for(j = 0; j < cm1; j++) {
      if(m1[i * ldm1 + j] != m2[i * ldm2 + j]) {
        printf("Discrepance in %d,%d: %.8lf , %.8lf\n", i, j, m1[i * ldm1 + j], m2[i * ldm2 + j]);
        return;
      }
    }
}

int main(int argc, char *argv[]) {
  int nodo, np, i, j, fa, fal, ca, lda, fb, cb, ldb, fc, fcl, cc, ldc, N;
  int long_name;
  double ti, tf;
  double *a, *b, *c, *c0;
  char    nombre_procesador[MPI_MAX_PROCESSOR_NAME];
  MPI_Status estado;
 
  MPI_Init(&argc, &argv);
  MPI_Comm_size(MPI_COMM_WORLD, &np);
  MPI_Comm_rank(MPI_COMM_WORLD, &nodo);
  MPI_Get_processor_name(nombre_procesador, &long_name);

// Se ejecuta con mpirun -np numeroprocesos ejecutable tamañomatriz

  if (nodo == 0) {
    N = atoi(argv[1]);
  }
  MPI_Bcast(&N, 1, MPI_INT, 0, MPI_COMM_WORLD);
  
  fa = ca = lda = fb = cb = ldb = fc = cc = ldc = N;
  fal = N / np;
  fcl = N / np;
  if (nodo == 0) {
    a = (double *) malloc(sizeof(double) * fa * ca);
    b = (double *) malloc(sizeof(double) * fb * cb);
    c = (double *) malloc(sizeof(double) * fc * cc);
  } else {
    a = (double *) malloc(sizeof(double) * fal * ca);
    b = (double *) malloc(sizeof(double) * fb * cb);
    c = (double *) malloc(sizeof(double) * fcl * cc);
  }
  
  if (nodo == 0) {
    c0 = (double *) malloc(sizeof(double) * fc * cc);
    initialize(a, fa, ca, lda);
    initialize(b, fb, cb, ldb);

    mms(a, fa, ca, lda, b, fb, cb, ldb, c0, fc, cc, ldc);
  }

  MPI_Barrier(MPI_COMM_WORLD);

  ti = MPI_Wtime();

  mm(a, fa, ca, lda, b, fb, cb, ldb, c, fc, cc, ldc, nodo, np);

  MPI_Barrier(MPI_COMM_WORLD);
  tf = MPI_Wtime();
  if (nodo == 0) {
    //printf("(%d) Process %d, %s, Time %.6lf\n", N, np, nombre_procesador, tf - ti);
    printf("%d\t%f\n", N, tf - ti);  
  }
  
  free(a);
  free(b);
  free(c);
  if (nodo == 0)
    free(c0);
  MPI_Finalize();
}

Writing mm-mpi.c


#### execução

In [ ]:
!mpicc mm-mpi.c -o mm-mpi
!mpirun --allow-run-as-root -np 4 ./mm-mpi 100

100	0.008182


### Implementação da multiplicação de matrizes com o OpenMP:

#### código

In [ ]:
%%writefile mm-openmp.c

/*
File:           mm-openmp.c
Version:        Solution
Purpose:        Matrix Multiply Sequential Algorithm in C using OpenMP
Author:         Murilo Boratto  <muriloboratto 'at' fieb.org.br>
Usage:
Hotocompile:   gcc mm-openmp.c -o mm -fopenmp
Hotoexecute:   OMP_NUM_THREADS=<threads> ./mm <size>
               OMP_NUM_THREADS=4         ./mm  100  
*/

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

void initializeMatrix(int *matrix, int size)
{
  for (int i = 0; i < size; i++)
    for (int j = 0; j < size; j++)
      matrix[i * size + j] = rand() % (10 - 1) * 1;
}

void printMatrix(int *matrix, int size)
{
  for (int i = 0; i < size; i++)
  {
    for (int j = 0; j < size; j++)
      printf("%d\t", matrix[i * size + j]);
    printf("\n");
  }
  printf("\n");
}

int main (int argc, char **argv)
{

 int size = atoi(argv[1]);  
 int i, j, k;
 double t1, t2;

 int  *A = (int *) malloc (sizeof(int)*size*size);
 int  *B = (int *) malloc (sizeof(int)*size*size);
 int  *C = (int *) malloc (sizeof(int)*size*size);

 initializeMatrix(A, size);
 initializeMatrix(B, size);

 t1 = omp_get_wtime();

 #pragma omp parallel for private(i, j, k)
   for(i = 0; i < size; i++)
    for(j = 0; j < size; j++)
      for(k = 0; k < size; k++)
        C[i * size + j] += A[i * size + k] * B[k * size + j];

 t2 = omp_get_wtime();

 printf("%d\t%f\n",size, t2-t1);

// printMatrix(A,size);
// printMatrix(B,size);
// printMatrix(C,size);

 return 0;

}


Writing mm-openmp.c


#### execução

In [ ]:
!chmod 777 mm-openmp.c

In [ ]:
!gcc mm-openmp.c -o mm-openmp -fopenmp
!OMP_NUM_THREADS=16 ./mm-openmp 100

100	0.005343


### Implementação da multiplicação de matrizes com a abordagem híbrida:

#### código

In [ ]:
%%writefile mm-mpi+openmp.c

/*
File:           mm-mpi+openmp.c
Purpose:        Parallelize matrix multiplication using OpenMP+MPI
Authors:        Francisco Almeida, Domingo Giménez, José Miguel Mantas, Antonio M. Vidal
Usage:
HowToCompile:   mpicc mm-mpi+openmp.c -o mm-mpi+openmp -fopenmp
HowToExecute:   mpirun -np <numberOfProcesses> ./mm-mpi+openmp <size> <threads> 
Example:        mpirun -np         4           ./mm-mpi+openmp  100       16     
Comments:
                ◆ Spanish code comments;          
*/

#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>
#include <mpi.h>
#include <omp.h>


void mm(double *a, int fa,int ca,int lda,double *b,int fb,int cb,int ldb,double *c,int fc,int cc,int ldc,int nodo,char *maquina)
{
  int i, j, k;
  double s;

#pragma omp parallel 
{
#pragma omp for private(i,j,k,s) schedule(static)
  for (i = 0; i < fa; i++) 
  {
    for(j=0;j<cb;j++)
    {
      s=0.;
      for (k = 0; k < ca; k++)
	      s = s+a[i*lda+k]*b[k*ldb+j];
      c[i*ldc+j]=s;
    }
  }
}
}

/*
c
c initialize - random initialization for array
c
*/

void initialize(double *m, int f,int c,int ld){
  int i, j;

  for (i = 0; i < f; i++)
  {
    for (j = 0; j < c; j++)
    {  
      m[i*ld+j] = (double)(i+j);
    }
  }
}

void initializealea(double *m, int f,int c,int ld){
  int i, j;

  for (i = 0; i < f; i++)
  {
    for (j = 0; j < c; j++)
    {  
      m[i*ld+j] = (double)rand()/RAND_MAX;
    }
  }
}

void escribir(double *m, int f,int c,int ld){
  int i, j;

  for (i = 0; i < f; i++)
  {
    for (j = 0; j < c; j++)
    {  
      printf("%.4lf ",m[i*ld+j]);
    }
    printf("\n");
  }
}
/*
c
c     mseconds - returns elapsed milliseconds since Jan 1st, 1970.
c
*/
long long mseconds(){
  struct timeval t;
  gettimeofday(&t, NULL);
  return t.tv_sec*1000 + t.tv_usec/1000;
}

void comparar(double *m1,int fm1,int cm1,int ldm1,double *m2,int fm2,int cm2,int ldm2)
{
  int i,j;

  for(i=0;i<fm1;i++)
    for(j=0;j<cm1;j++)
    {
      if(m1[i*ldm1+j]!=m2[i*ldm2+j])
      {
        printf("Discrepance in %d,%d: %.8lf , %.8lf\n",i,j,m1[i*ldm1+j],m2[i*ldm2+j]);
        return;
      }
    }
}

int main(int argc,char *argv[]) {
  int nodo,np,i, j,fa,fal,ca,lda,fb,cb,ldb,fc,fcl,cc,ldc,N,NUMTHREADS;
  int long_name;
  double ti,tf;
  double *a,*b,*c,*c0;
  char    nombre_procesador[MPI_MAX_PROCESSOR_NAME];
  MPI_Status estado;
 
  MPI_Init(&argc,&argv);
  MPI_Comm_size(MPI_COMM_WORLD,&np);
  MPI_Comm_rank(MPI_COMM_WORLD,&nodo);
  MPI_Get_processor_name(nombre_procesador,&long_name);

  if(nodo==0)
  {
    N=atoi(argv[1]);
    NUMTHREADS=atoi(argv[2]);
  }
  MPI_Bcast(&N,1,MPI_INT,0,MPI_COMM_WORLD);
  MPI_Bcast(&NUMTHREADS,1,MPI_INT,0,MPI_COMM_WORLD);
  omp_set_num_threads(NUMTHREADS);
  
  fa=ca=lda=fb=cb=ldb=fc=cc=ldc=N;
  fal=N/np;
  fcl=N/np;
  if(nodo==0)
  {
    a = (double *) malloc(sizeof(double)*fa*ca);
    b = (double *) malloc(sizeof(double)*fb*cb);
    c = (double *) malloc(sizeof(double)*fc*cc);
  }
  else
  {
    a = (double *) malloc(sizeof(double)*fal*ca);
    b = (double *) malloc(sizeof(double)*fb*cb);
    c = (double *) malloc(sizeof(double)*fcl*cc);
  }
  
  if(nodo==0)
  {
    c0=(double *) malloc(sizeof(double)*fc*cc);
    initialize(a,fa,ca,lda);
    for(i=1;i<np;i++)
    {
      MPI_Send(&a[i*lda*N/np],fal*ca,MPI_DOUBLE,i,20,MPI_COMM_WORLD);
    }
    initialize(b,fb,cb,ldb);
    MPI_Bcast(b,fb*cb,MPI_DOUBLE,0,MPI_COMM_WORLD);
    mm(a,fa,ca,lda,b,fb,cb,ldb,c0,fc,cc,ldc,nodo,nombre_procesador);
  }
  else
  {
    MPI_Recv(a,fal*ca,MPI_DOUBLE,0,20,MPI_COMM_WORLD,&estado);
    MPI_Bcast(b,fb*cb,MPI_DOUBLE,0,MPI_COMM_WORLD);
  } 

  MPI_Barrier(MPI_COMM_WORLD);

  ti=MPI_Wtime();

  mm(a,fal,ca,lda,b,fb,cb,ldb,c,fcl,cc,ldc,nodo,nombre_procesador);

  MPI_Barrier(MPI_COMM_WORLD);
  tf=MPI_Wtime();
  if(nodo==0)
  {
    //printf("(%d) Threads %d, Process %d, %s, Time %.6lf\n\n",N, NUMTHREADS, np, nombre_procesador,tf-ti);
    printf("%d\t%f\n", N, tf-ti);
    for(i=1;i<np;i++)
    {
      MPI_Recv(&c[i*ldc*N/np],fcl*cc,MPI_DOUBLE,i,30,MPI_COMM_WORLD,&estado);
    }
  }
  else
  {
    MPI_Send(c,fcl*cc,MPI_DOUBLE,0,30,MPI_COMM_WORLD);
  } 
  
  free(a);
  free(b);
  free(c);
  if(nodo==0)
    free(c0);
  MPI_Finalize();
}

Writing mm-mpi+openmp.c


#### execução

In [ ]:
!mpicc mm-mpi+openmp.c -o mm-mpi+openmp -fopenmp
!mpirun --allow-run-as-root -np 4 ./mm-mpi+openmp 1000 16

1000	8.321745


### Script bash para execução das implementações anteriores e geração dos gráficos comparativos


#### código

In [ ]:
%%writefile script.sh
#!/bin/sh

export LC_NUMERIC="en_US.UTF-8"

set -euo pipefail

clear 

###################################
# FUNCTIONS                       #
###################################

showPropeller() {
   
   tput civis
   
   while [ -d /proc/$! ]
   do
      for i in / - \\ \|
      do
         printf "\033[1D$i"
         sleep .1
      done
   done
   
   tput cnorm
}

create_plot_script_time() {
cat <<EOF >time.plt
set title "Execution Time" 
set ylabel "Time (Seconds)"
set xlabel "Size"

set style line 1 lt 2 lc rgb "cyan"   lw 2 
set style line 2 lt 2 lc rgb "red"    lw 2
set style line 3 lt 2 lc rgb "gold"   lw 2
set style line 4 lt 2 lc rgb "green"  lw 2
set style line 5 lt 2 lc rgb "blue"   lw 2
set style line 6 lt 2 lc rgb "black"  lw 2
set terminal postscript eps enhanced color
set output 'time.png'

set xtics nomirror
set ytics nomirror
set key top left
set key box
set style data lines

plot "file_comparison.data" using 1:2 title "Sequential"              ls 1 with linespoints,\
     "file_comparison.data" using 1:3 title "T=2"                     ls 2 with linespoints,\
     "file_comparison.data" using 1:4 title "T=3"                     ls 3 with linespoints,\
     "file_comparison.data" using 1:5 title "T=4"                     ls 4 with linespoints
EOF
}

create_plot_script_speedup() {
cat <<EOF >speedup.plt
set title "Speedup" 
set ylabel "Speedup"
set xlabel "Size"

set style line 1 lt 2 lc rgb "cyan"   lw 2 
set style line 2 lt 2 lc rgb "red"    lw 2
set style line 3 lt 2 lc rgb "gold"   lw 2
set style line 4 lt 2 lc rgb "green"  lw 2
set style line 5 lt 2 lc rgb "blue"   lw 2
set style line 6 lt 2 lc rgb "black"  lw 2
set terminal postscript eps enhanced color
set output 'speedup.png'

set xtics nomirror
set ytics nomirror
set key top left
set key box
set style data lines

plot "file_speedup.data" using 1:2 title "T=2"    ls 1 with linespoints,\
     "file_speedup.data" using 1:3 title "T=3"    ls 2 with linespoints,\
     "file_speedup.data" using 1:4 title "T=4"    ls 3 with linespoints
EOF
}

################################################
# 0. COMPILATION + PERMISSIONS  TO EXECUTE     #
################################################

compile_and_execute_openmp() {
    # module load gcc/11.1.0
gcc mm-openmp.c -o mm -fopenmp -O3
chmod +x mm

###################################
# Experimental Times              #
###################################

sleep 5 > /dev/null 2>&1 &

printf "Loading...\040\040" ; showPropeller
echo " "

for i in 100 200 300 400 500 600 700 800 900 1000
do
printf "\033[1D$i :" 

for (( j=1; j<=4; j+=1 ))
do
OMP_NUM_THREADS=$j   ./mm               "$i"    >> "file$j"
done
done

clear 
}

compile_and_execute_mpi() {
mpicc mm-mpi.c -o mm
chmod +x mm


###################################
# Experimental Times              #
###################################

sleep 5 > /dev/null 2>&1 &

printf "Loading...\040\040" ; showPropeller
echo " "

for i in 100 200 300 400 500 600 700 800 900 1000
do
printf "\033[1D$i :" 
for j in {1..4}
do
mpirun -np "$j" ./mm    "$i"    >> "file$j"
done
done

clear 
}

compile_and_execute_openmpi() {
mpicc mm-mpi+openmp.c -o mm -fopenmp
chmod +x mm


###################################
# Experimental Times              #
###################################

sleep 5 > /dev/null 2>&1 &

printf "Loading...\040\040" ; showPropeller
echo " "

for i in 100 200 300 400 500 600 700 800 900 1000
do
printf "\033[1D$i :" 
for j in {1..4}
do
mpirun -np "$j" ./mm    "$i"    $j*$j >> "file$j"
done

done

clear 
}

time_comparison() {
echo " "
echo " "
echo "    ********************************"
echo "    * Experimental Time Comparison *"
echo "    ********************************"
echo " "

pr -m -t -s\  file1 file2 file3 file4 | awk '{print $1,"\t",$2,"\t",$4,"\t",$6,"\t",$8}' >> file_comparison.data

echo " "
echo "    [#][size]       [S]	           [T02]	   [T03]	  [T04]"
cat -n  file_comparison.data

sleep 3

#####################
# SPEEDUP           #
#####################

awk '{print $1, " ",(($2*1000)/($3*1000))}' file_comparison.data >> fspeed0 #OMP T=02 
awk '{print $1, " ",(($2*1000)/($4*1000))}' file_comparison.data >> fspeed1 #OMP T=03
awk '{print $1, " ",(($2*1000)/($5*1000))}' file_comparison.data >> fspeed2 #OMP T=04

pr -m -t -s\  fspeed0 fspeed1 fspeed2| awk '{print $1,"\t",$2,"\t",$4,"\t",$6,"\t",$8}' >> file_speedup.data

# pr -m -t -s\  fspeed0 fspeed1 fspeed2 fspeed3 fspeed4 fspeed5 fspeed6 fspeed7 fspeed8 fspeed9 fspeed10| awk '{print $1,"\t",$2,"\t",$4,"\t",$6,"\t",$8,"\t",$10,"\t",$12,"\t",$14}' >> file_speedup.data

echo " "
echo " "
echo "    ********************************"
echo "    * Speedup  Rate                *"
echo "    ********************************"
echo " "
echo "    [#][size]    [ST02]	          [ST03]	  [ST04]"
cat -n file_speedup.data
}


ploting_setup() {
#####################
# PLOTING           #
#####################
echo " "
echo "Do you want to plot a graphic (y/n)?"
read resp

if [[ $resp = "y" ]];then
         echo "ploting eps graphic with gnuplot..."
         create_plot_script_time
         create_plot_script_speedup
         gnuplot "time.plt"
         gnuplot "speedup.plt"
#rename the files
  filename=$1
  mv time.png  time_$filename.png
  mv speedup.png  speedup_$filename.png

fi
}

remove_unnecessary_files() {
echo " "
echo "[Remove unnecessary files] "
rm -f *.txt file* fspeed* *.data mm *.plt
echo " "

sleep 7 > /dev/null 2>&1 &

printf "Loading...\040\040" ; showPropeller
echo " "
echo "[END] " 
echo " "
}



#  execution

echo "executing OpenMP"
compile_and_execute_openmp
time_comparison 
ploting_setup "openmp"
remove_unnecessary_files


echo "executing MPI"
compile_and_execute_mpi
time_comparison
ploting_setup "mpi"
remove_unnecessary_files


echo "executing OpenMP+MPI"
compile_and_execute_openmpi
time_comparison 
ploting_setup "openmp_mpi"
remove_unnecessary_files

Writing script.sh


#### execução

In [ ]:
!bash script.sh

 
[Remove unnecessary files] 
 
Loading...  

### Visualização dos resultados encontrados

Tamanho do problema e tempo de execução


Após a execução da multiplicação de matrizes utilizando OpenMP, MPI e uma abordagem híbrida, geramos os gráficos correspondentes. Consideramos uma variação no tamanho do problema de 100 a 1000, além de um número de *threads* variando de 2 a 16 e execuções por troca de mensagens com 4 processos.

Na Figura 1, observa-se a relação entre o tempo de execução e o tamanho do problema, considerando direntes quantidades de threads. Percebe-se que o tempo da execução sequencial é maior que nas execuções paralelizadas, em que 4 threads mostrou-se um número satisfatório para um tamanho de problema inferior a 800. Contudo, a mudança drástica no tempo de execução com 4 threads, aproximando seu desempenho da execução sequencial pode indicar uma interferência dos processos já em execução máquina utilizada para os testes.
<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/time_openmp.png?raw=true" />
</p>



**Figura 1.** *Comparação do tempo de execução da multiplicação de matrizes de diferentes tamanhos, utilizando OpenMP*

Já na implementação utilizando o MPI, a partir do gráfico visto na Figura 2, percebe-se que o tempo de execução sequencial também mostrou-se maior que o da implementação paralelizada, em que a utilização de 3 ou 4 processos apresenta-se mais adequada a depender do tamanho do problema.

<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/time_mpi.png?raw=true" />
</p>

**Figura 2.**  *Comparação do tempo de execução da multiplicação de matrizes de diferentes tamanhos, utilizando MPI*

É  importante ressaltar que ao comparar os melhores tempos de execução da multiplicação de matrizes da abordagem com OpenMP (Figura 1) e com MPI (Figura 2), é perceptível que a execução com *threads* tem um tempo bem menor que a execução com troca de mensagens.

Já na execução com o modelo híbrido a quantidade de 3 processos para 9 *threads* se destaca em relação a ter o menor tempo de execução para os maiores valores no tamanho do problema.
 
<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/time_openmpi.png?raw=true" />
</p>

**Figura 3.** *Comparação do tempo de execução da multiplicação de matrizes de diferentes tamanhos, utilizando OpenMP e MPI*

#### Speedup


Ao realizar a multiplicação de matrizes com o OpenMP, foi possível obter o grafico da Figura 4, que mostra a relação entre o tamanho do problema e o speedup para diferentes números de threads. Assim, é possível observar que os melhores resultados foram os encontrados para 3 e 4 threads, a depender do tamanho do problema.

<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/speedup_openmp.png?raw=true" />
</p>

**Figura 4.** *Comparação do speedup entre a quantidade de threads utilizadas, com OpenMP.*

Já ao observar o *speedup* com a utilizado do MPI, percebe-se que continua a variação entre os valores de 3 e 4, nesse caso de processos.
<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/speedup_mpi.png?raw=true"/>
</p>

**Figura 5.** *Comparação do speedup entre a quantidade de threads utilizadas, com MPI.*

Visualizando o gráfico de *speedup* na Figura 6 para o modelo híbrido, observa-se mais estabilidade em relação aos *speedups* apresentados nas Figuras 4 e 5 com os outros paradigmas. Entretanto, a quantidade de 3 processos com 9 *threads* é a que se mantém com maior *speedup* dentro da variação do tamanho do problema.

<p align="center">
<img src="https://github.com/AmadoMaria/hands-on-supercomputing-with-parallel-computing/blob/master/Hands-On-5-Basic-Hybrid-Application-MPI+OpenMP/material/speedup_openmpi.png?raw=true" />
</p>

**Figura 6.** *Comparação do tempo de execução da multiplicação de matrizes de diferentes tamanhos, utilizando OpenMP e MPI*

# Considerações Finais

A presente prática teve como objetivo comparar as estratégias de otimização de um algoritmo de multiplicação de matrizes, considerando a aplicação dos paradigmas, utilizando os modelos com OpenMP, MPI e híbrido. A paralelização de um código, seja através da troca de mensagens ou da criação de threads tem um grande impacto na otimização dos algoritmos. Contudo, a implementação da multiplicação de matrizes com o OpenMP apresentou um *speedup* maior em relação a execução com MPI. Além disso, a versão do código que utilizou ambos os recursos apresentou um tempo de execução intermediário quando comparado as demais, o que mostra uma otimização em relação ao tempo sequencial, mas também evidencia o custo da troca de mensagens quando consideramos a paralelização apenas com *threads*.
Dessa forma, a abordagem híbrida mostrou-se uma boa alternativa para o maior aproveitamento dos recursos computacionais disponíveis, melhorando tanto o uso da memória compartilhada quando distribuída. 

# Referências

[1] Robinson, S. (2005). Toward an optimal algorithm for matrix multiplication. SIAM news, 38(9), 1-3.

[2] DE ARAUJO, L. R.; WEBER, C. M.; PUNTEL, F. E.; CHARÃO, A. S.; LIMA, J. V. F. Análise Comparativa de MPI e OpenMP em Implementações de Transposição de Matrizes para Arquitetura com Memória Compartilhada. Revista Eletrônica de Iniciação Científica em Computação, [S. l.], v. 16, n. 5, 2018. DOI: 10.5753/reic.2018.1072. Disponível em: https://sol.sbc.org.br/journals/index.php/reic/article/view/1072. Acesso em: 7 out. 2022.

[3] Karniadakis, G., & Kirby II, R. (2003). Parallel Scientific Computing in C and MPI: A Seamless Approach to Parallel Algorithms and their Implementation. Cambridge: Cambridge University Press. doi:10.1017/CBO9780511812583


[4] MENDES, Leonardo; FERREIRA, Eduardo. Programação Paralela Híbrida aplicada em um Cluster Computacional com testes em Multiplicação de Matrizes. In: ESCOLA REGIONAL DE ALTO DESEMPENHO DA REGIÃO SUL (ERAD-RS), 20. , 2020, Santa Maria. Anais [...]. Porto Alegre: Sociedade Brasileira de Computação, 2020 . p. 89-92. ISSN 2595-4164. DOI: https://doi.org/10.5753/eradrs.2020.10763.